reg children educ7 age agesq evermarr urban electric tv

In [21]:
import pandas as pd
import numpy as np
df = pd.read_csv('/Users/pranjal/Desktop/Causal-Inference/data/fertil2.csv')

In [22]:
df.head()

,mnthborn,yearborn,age,electric,radio,tv,bicycle,educ,ceb,agefbrth,...,agesq,urban,urb_educ,spirit,protest,catholic,frsthalf,educ0,evermarr,educ7
0,5,64,24,1.0,1.0,1.0,1.0,12,0,NaN,...,576,1,12,0,0,0,1,0,0,1
1,1,56,32,1.0,1.0,1.0,1.0,13,3,25.0,...,1024,1,13,0,0,0,1,0,1,1
2,7,58,30,1.0,0.0,0.0,0.0,5,1,27.0,...,900,1,5,1,0,0,0,0,1,0
3,11,45,42,1.0,0.0,1.0,0.0,4,3,17.0,...,1764,1,4,0,0,0,0,0,1,0
4,5,45,43,1.0,1.0,1.0,1.0,11,2,24.0,...,1849,1,11,0,1,0,1,0,1,1


In [23]:
cat = df.select_dtypes('object').columns
df = pd.get_dummies(df, columns = cat, drop_first = True)

In [25]:
df.head()

,mnthborn,yearborn,age,electric,radio,tv,bicycle,educ,ceb,agefbrth,...,agesq,urban,urb_educ,spirit,protest,catholic,frsthalf,educ0,evermarr,educ7
0,5,64,24,1.0,1.0,1.0,1.0,12,0,NaN,...,576,1,12,0,0,0,1,0,0,1
1,1,56,32,1.0,1.0,1.0,1.0,13,3,25.0,...,1024,1,13,0,0,0,1,0,1,1
2,7,58,30,1.0,0.0,0.0,0.0,5,1,27.0,...,900,1,5,1,0,0,0,0,1,0
3,11,45,42,1.0,0.0,1.0,0.0,4,3,17.0,...,1764,1,4,0,0,0,0,0,1,0
4,5,45,43,1.0,1.0,1.0,1.0,11,2,24.0,...,1849,1,11,0,1,0,1,0,1,1


In [26]:
outcome = 'children'
treatment = 'educ7'
rest = list(df.drop([outcome, treatment], axis = 1).columns)
#rest = ['age', 'evermarr', 'urban', 'electric', 'tv']
df = df[[outcome] + [treatment] + rest]

In [27]:
#import wooldridge
#df = wooldridge.data('jtrain3')
#df['avg'] = 0.5 * (df.re74+df.re75)
#df = df.dropna()
df = df.fillna(0)
#df = df[df.avg <= 15]
y = df[outcome]
d = df[treatment]
x = df[rest].astype('float')
print(df.shape)
df.head()

(4361, 28)


,children,educ7,mnthborn,yearborn,age,electric,radio,tv,bicycle,educ,...,heduc,agesq,urban,urb_educ,spirit,protest,catholic,frsthalf,educ0,evermarr
0,0,1,5,64,24,1.0,1.0,1.0,1.0,12,...,0.0,576,1,12,0,0,0,1,0,0
1,3,1,1,56,32,1.0,1.0,1.0,1.0,13,...,12.0,1024,1,13,0,0,0,1,0,1
2,1,0,7,58,30,1.0,0.0,0.0,0.0,5,...,7.0,900,1,5,1,0,0,0,0,1
3,2,0,11,45,42,1.0,0.0,1.0,0.0,4,...,11.0,1764,1,4,0,0,0,0,0,1
4,2,1,5,45,43,1.0,1.0,1.0,1.0,11,...,14.0,1849,1,11,0,1,0,1,0,1


In [28]:
# Simple Comparision of Means
import statsmodels.api as sm
mod = sm.OLS(y, sm.add_constant(np.c_[d], prepend=False))
res = mod.fit()
print(res.summary())
print(res.params[0])
print(res.bse[0])

                            OLS Regression Results                            
Dep. Variable:               children   R-squared:                       0.157
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     810.1
Date:                Mon, 05 Dec 2022   Prob (F-statistic):          1.39e-163
Time:                        01:46:07   Log-Likelihood:                -9297.7
No. Observations:                4361   AIC:                         1.860e+04
Df Residuals:                    4359   BIC:                         1.861e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -1.7701      0.062    -28.462      0.0

In [29]:
# Pooled Regression Adjustment
import statsmodels.api as sm
mod = sm.OLS(y, sm.add_constant(np.c_[d, x], prepend=False))
res = mod.fit()
print(res.summary())
print(res.params[0])
print(res.bse[0])

                            OLS Regression Results                            
Dep. Variable:               children   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.959
Method:                 Least Squares   F-statistic:                     3751.
Date:                Mon, 05 Dec 2022   Prob (F-statistic):               0.00
Time:                        01:46:09   Log-Likelihood:                -2706.0
No. Observations:                4361   AIC:                             5468.
Df Residuals:                    4333   BIC:                             5647.
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0170      0.025      0.682      0.4

In [30]:
import numpy as np
from doubleml.datasets import make_plr_CCDDHNR2018
from doubleml import DoubleMLData

np.random.seed(1234)
dml_data_bonus = DoubleMLData(df, y_col=outcome,
                                  d_cols=treatment,
                                  x_cols=list(rest))
print(dml_data_bonus)
from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
learner = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt', max_depth= 6)
ml_l_bonus = clone(learner)
ml_m_bonus = clone(learner)
learner = LassoCV()
ml_l_sim = clone(learner)
ml_m_sim = clone(learner)
def non_orth_score(y, d, l_hat, m_hat, g_hat, smpls):
    u_hat = y - g_hat
    psi_a = -np.multiply(d, d)
    psi_b = np.multiply(d, u_hat)
    return psi_a, psi_b

from doubleml import DoubleMLPLR
np.random.seed(3141)
obj_dml_plr_bonus = DoubleMLPLR(dml_data_bonus, ml_l_bonus, ml_m_bonus)
obj_dml_plr_bonus.fit();
print(obj_dml_plr_bonus)

================== DoubleMLData Object ==================

------------------ Data summary      ------------------
Outcome variable: children
Treatment variable(s): ['educ7']
Covariates: ['mnthborn', 'yearborn', 'age', 'electric', 'radio', 'tv', 'bicycle', 'educ', 'ceb', 'agefbrth', 'knowmeth', 'usemeth', 'monthfm', 'yearfm', 'agefm', 'idlnchld', 'heduc', 'agesq', 'urban', 'urb_educ', 'spirit', 'protest', 'catholic', 'frsthalf', 'educ0', 'evermarr']
Instrument variable(s): None
No. Observations: 4361

------------------ DataFrame info    ------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4361 entries, 0 to 4360
Columns: 28 entries, children to evermarr
dtypes: float64(12), int64(16)
memory usage: 954.1 KB

================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: children
Treatment variable(s): ['educ7']
Covariates: ['mnthborn', 'yearborn', 'age', 'electric', 'radio', 'tv', 'bicycle', 'educ', 'c

In [31]:
import numpy as np
from doubleml.datasets import make_plr_CCDDHNR2018
from doubleml import DoubleMLData

np.random.seed(1234)
dml_data_bonus = DoubleMLData(df, y_col=outcome,
                                  d_cols=treatment,
                                  x_cols=list(rest))
print(dml_data_bonus)
from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
learner = RandomForestRegressor(n_estimators = 100, max_features = 'sqrt', max_depth= 6)
ml_l_bonus = clone(learner)
ml_m_bonus = clone(learner)
learner = LassoCV()
ml_l_sim = clone(learner)
ml_m_sim = clone(learner)
def non_orth_score(y, d, l_hat, m_hat, g_hat, smpls):
    u_hat = y - g_hat
    psi_a = -np.multiply(d, d)
    psi_b = np.multiply(d, u_hat)
    return psi_a, psi_b

from doubleml import DoubleMLPLR
np.random.seed(3141)
obj_dml_plr_bonus = DoubleMLPLR(dml_data_bonus, ml_l_bonus, ml_m_bonus)
obj_dml_plr_bonus.fit();
print(obj_dml_plr_bonus)

================== DoubleMLData Object ==================

------------------ Data summary      ------------------
Outcome variable: children
Treatment variable(s): ['educ7']
Covariates: ['mnthborn', 'yearborn', 'age', 'electric', 'radio', 'tv', 'bicycle', 'educ', 'ceb', 'agefbrth', 'knowmeth', 'usemeth', 'monthfm', 'yearfm', 'agefm', 'idlnchld', 'heduc', 'agesq', 'urban', 'urb_educ', 'spirit', 'protest', 'catholic', 'frsthalf', 'educ0', 'evermarr']
Instrument variable(s): None
No. Observations: 4361

------------------ DataFrame info    ------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4361 entries, 0 to 4360
Columns: 28 entries, children to evermarr
dtypes: float64(12), int64(16)
memory usage: 954.1 KB

================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: children
Treatment variable(s): ['educ7']
Covariates: ['mnthborn', 'yearborn', 'age', 'electric', 'radio', 'tv', 'bicycle', 'educ', 'c

In [32]:
# DML regression - still yeilds unbiased estimate of ATE 
from econml.dml import LinearDML
est = LinearDML(random_state=45)
est.fit(y, d, X=None,W=x)
est.summary()

Coefficient Results:  X is None, please call intercept_inference to learn the constant!


,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,-0.055,0.034,-1.634,0.102,-0.122,0.011
